## Convert data

Obtain data from https://satijalab.org/seurat/articles/multimodal_vignette.html

Install R dependencies:
```R
install.packages("seurat")
install.packages("devtools")
devtools::install_github("mojaveazure/seurat-disk")
devtools::install_github("vitessce/vitessceR")
```

```R
library(Seurat)

# Load in the RNA UMI matrix

# Note that this dataset also contains ~5% of mouse cells, which we can use as negative
# controls for the protein measurements. For this reason, the gene expression matrix has
# HUMAN_ or MOUSE_ appended to the beginning of each gene.
cbmc.rna <- as.sparse(read.csv(file = "/Users/mkeller/Downloads/GSE100866_CBMC_8K_13AB_10X-RNA_umi.csv.gz", sep = ",",
                               header = TRUE, row.names = 1))

# To make life a bit easier going forward, we're going to discard all but the top 100 most
# highly expressed mouse genes, and remove the 'HUMAN_' from the CITE-seq prefix
cbmc.rna <- CollapseSpeciesExpressionMatrix(cbmc.rna)

# Load in the ADT UMI matrix
cbmc.adt <- as.sparse(read.csv(file = "/Users/mkeller/Downloads/GSE100866_CBMC_8K_13AB_10X-ADT_umi.csv.gz", sep = ",",
                               header = TRUE, row.names = 1))

# Note that all operations below are performed on the RNA assay Set and verify that the
# default assay is RNA
DefaultAssay(cbmc) <- "RNA"
DefaultAssay

# perform visualization and clustering steps
cbmc <- NormalizeData(cbmc)
cbmc <- FindVariableFeatures(cbmc)
cbmc <- ScaleData(cbmc)
cbmc <- RunPCA(cbmc, verbose = FALSE)
cbmc <- FindNeighbors(cbmc, dims = 1:30)
cbmc <- FindClusters(cbmc, resolution = 0.8, verbose = FALSE)
cbmc <- RunUMAP(cbmc, dims = 1:30)
DimPlot(cbmc, label = TRUE)

# Normalize ADT data,
DefaultAssay(cbmc) <- "ADT"
cbmc <- NormalizeData(cbmc, normalization.method = "CLR", margin = 2)
DefaultAssay(cbmc) <- "RNA"

# Note that the following command is an alternative but returns the same result
cbmc <- NormalizeData(cbmc, normalization.method = "CLR", margin = 2, assay = "ADT")


vitessceR::seurat_to_anndata_zarr(cbmc, file.path("data", "multimodal_vignette.rna.h5ad.zarr"), assay = "RNA")
vitessceR::seurat_to_anndata_zarr(cbmc, file.path("data", "multimodal_vignette.adt.h5ad.zarr"), assay = "ADT")
```

In [ ]:
import scanpy as sc
import numpy as np
import scipy.cluster
from scipy import sparse

from anndata import read_zarr
from os.path import join

from vitessce import (
    VitessceConfig,
    ViewType as vt,
    CoordinationType as ct,
    FileType as ft,
    AnnDataWrapper,
    OmeTiffWrapper,
)
from vitessce.data_utils import (
    optimize_adata,
    to_dense,
    VAR_CHUNK_SIZE,
    sort_var_axis,
)

In [ ]:
rna_zarr = join("data", "multimodal_vignette.rna.h5ad.zarr")
adt_zarr = join("data", "multimodal_vignette.adt.h5ad.zarr")

In [ ]:
rna_adata = read_zarr(rna_zarr)
rna_adata

In [ ]:
adt_adata = read_zarr(adt_zarr)
adt_adata

## Configure visualization

In [ ]:
vc = VitessceConfig(schema_version="1.0.15", name='Multimodal seurat object', description='RNA+ADT')

In [ ]:
dataset = vc.add_dataset(name='RNA+ADT').add_object(AnnDataWrapper(
    adata_path=rna_zarr,
    obs_embedding_paths=["obsm/X_umap", "obsm/X_pca"],
    obs_embedding_names=["UMAP", "PCA"],
    obs_set_paths=["obs/seurat_clusters"],
    obs_set_names=["Seurat Clusters"],
    obs_feature_matrix_path="X",
    coordination_values={
        "featureType": "gene",
        "featureValueType": "expression"
    }
)).add_object(AnnDataWrapper(
    adata_path=adt_zarr,
    obs_embedding_paths=["obsm/X_umap"],
    obs_embedding_names=["UMAP"],
    obs_set_paths=["obs/seurat_clusters"],
    obs_set_names=["Seurat Clusters"],
    obs_feature_matrix_path="X",
    coordination_values={
        "featureType": "tag",
        "featureValueType": "count"
    }
))

In [ ]:
umap_scatterplot_by_rna = vc.add_view(vt.SCATTERPLOT, dataset=dataset, mapping="UMAP")
umap_scatterplot_by_adt = vc.add_view(vt.SCATTERPLOT, dataset=dataset, mapping="UMAP")

gene_list = vc.add_view(vt.FEATURE_LIST, dataset=dataset)
protein_list = vc.add_view(vt.FEATURE_LIST, dataset=dataset)

rna_heatmap = vc.add_view(vt.HEATMAP, dataset=dataset).set_props(transpose=True)
adt_heatmap = vc.add_view(vt.HEATMAP, dataset=dataset).set_props(transpose=True)

In [ ]:
coordination_types = [ct.FEATURE_TYPE, ct.FEATURE_VALUE_TYPE, ct.FEATURE_SELECTION, ct.OBS_COLOR_ENCODING, ct.FEATURE_VALUE_COLORMAP_RANGE]
vc.link_views([umap_scatterplot_by_rna, gene_list, rna_heatmap], coordination_types, ["gene", "expression", None, 'cellSetSelection', [0.0, 0.3]])
vc.link_views([umap_scatterplot_by_adt, protein_list, adt_heatmap], coordination_types, ["tag", "count", None, 'cellSetSelection', [0.0, 1.0]])

In [ ]:
vc.layout(
    (rna_heatmap | (umap_scatterplot_by_rna | gene_list))
    / (adt_heatmap | (umap_scatterplot_by_adt | protein_list))
);

In [ ]:
vw = vc.widget()
vw